---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

a = pd.read_csv('City_Zhvi_AllHomes.csv')
b = pd.read_table('university_towns.txt',names = ['teste'])
c = pd.read_excel('gdplev.xls', usecols = [4, 6], skiprows=7, names = ['Quarter', 'GDP'])

In [4]:
def get_list_of_university_towns():
    d = pd.DataFrame(b['teste'].str.split('(').str.get(0), columns=['teste'])
    d.rename(columns={'teste':'RegionName'}, inplace = True)
    d['RegionName'][11].find('[edit]')
    d['RegionName'] = d['RegionName'].str.strip(' ')
    d['State'] = ''
    for i in d.index:
        if d['RegionName'][i].find('[edit]') != -1:
            k = d['RegionName'][i].split('[')[0]
            d['State'][i] = k
        d['State'][i] = k

    d = d.drop(d[d['RegionName'].str.contains('edit')].index)
    d = d[['State', 'RegionName']]
    
    return d
get_list_of_university_towns()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
10,Alaska,Fairbanks
12,Arizona,Flagstaff


In [5]:
def get_recession_start():
    import pandas as pd
    c = pd.read_excel('gdplev.xls', usecols = [4, 6], skiprows=7, names = ['Quarter', 'GDP'])
    ini_rec = ''
    c['year'] = c['Quarter'].str.split('q').str.get(0)    
    c = c.loc[c['year'] >= '2000']
    c.reset_index(inplace = True)
    for i in c.index:
        if c['GDP'][i+2] < c['GDP'][i+1]:
            if c['GDP'][i+1] < c['GDP'][i]:
                ini_rec = c['Quarter'][i+1]
                break
            else:
                continue
            break
        else:
            continue
        
    return ini_rec
get_recession_start()

'2008q3'

In [6]:
def get_recession_end():
    import pandas as pd
    c = pd.read_excel('gdplev.xls', usecols = [4, 6], skiprows=7, names = ['Quarter', 'GDP'])
    end_rec = ''
    for i in (c.index + c.loc[c['Quarter'] == get_recession_start()].index[0]):
        if c['GDP'][i+2] > c['GDP'][i+1]:
            if c['GDP'][i+1] > c['GDP'][i]:
                end_rec = c['Quarter'][i+2]
                break
            else:
                continue
            break
        else:
            continue
        
    return end_rec
get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    index_start = c.loc[c['Quarter'] == get_recession_start()].index[0]
    index_end = c.loc[c['Quarter'] == get_recession_end()].index[0]
    gdp_min = c['GDP'][index_start : index_end].min()
    bottom_rec = c['Quarter'][c.loc[c['GDP'] == gdp_min].index[0]]
    return bottom_rec
get_recession_bottom()

'2009q2'

In [5]:
def convert_housing_data_to_quarters():
    import pandas as pd
    a = pd.read_csv('City_Zhvi_AllHomes.csv')
    e = a.copy()
    k = 1
    for i in range(51,249,3):
        e[e.iloc[:,i].name[:4] + 'q' + str(k)] = (e.iloc[:,i] + e.iloc[:,i+1] + e.iloc[:,i+2])/3
        k = k + 1
        if k > 4:
            k = 1
    i = 249
    e[e.iloc[:,i].name[:4] + 'q' + str(k)] = (e.iloc[:,i] + e.iloc[:,i+1] )/2

    e.drop(e.columns[3:251], axis=1, inplace = True)
    e.drop(e.columns[0], axis=1, inplace = True)
        
    f = pd.DataFrame([states.values()]).T
    f['Abrev'] = pd.DataFrame([states.keys()]).T
    f.rename(columns={0:'State'}, inplace = True)
    g = pd.merge(get_list_of_university_towns(), f, how='left', left_on='State', right_on='State')
    g.rename(columns={'State':'State_ext','Abrev':'State'}, inplace = True)
    g.drop(g.columns[1], axis=1, inplace = True)
    g = g.drop_duplicates()
    
    e.set_index('State', inplace = True)
    g.set_index('State', inplace = True)
    h = e.join(g)
    h.rename(columns={'State_ext':'State'}, inplace = True)
    h.set_index(['State','RegionName'], inplace = True)
    
    return h
convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State         RegionName                                                      
Alaska        Anchorage         174633.333333  175266.666667  179566.666667   
              Fairbanks         163200.000000  165033.333333  169300.000000   
              Palmer            168766.666667  170566.666667  174233.333333   
              North Pole        157466.666667  159100.000000  162133.333333   
              Juneau            192466.666667  194300.000000  195166.666667   
              Kenai                       NaN            NaN            NaN   
              Soldotna                    NaN            NaN            NaN   
              Kodiak                      NaN            NaN            NaN   
              Tanaina           160500.000000  163500.000000  166366.666667   
              Ketchikan                   NaN            NaN            NaN   
              Lakes             172733.333333  174133.333333  177266.666667   
              Homer                       NaN            NaN            NaN   
              Seward                      NaN            NaN            NaN   
              Anchor Point                NaN            NaN            NaN   
Alabama       Mobile             78233.333333   79366.666667   80433.333333   
              Birmingham         54033.333333   54400.000000   54966.666667   
              Huntsville                  NaN            NaN            NaN   
              Tuscaloosa        104500.000000  106033.333333  108500.000000   
              Decatur                     NaN            NaN            NaN   
              Hoover            184966.666667  186366.666667  187200.000000   
              Madison                     NaN            NaN            NaN   
              Northport         125866.666667  126366.666667  126633.333333   
              Wetumpka                    NaN            NaN            NaN   
              Fairhope          161400.000000  161933.333333  161633.333333   
              Albertville        73966.666667   72600.000000   72833.333333   
              Alabaster         122133.333333  123066.666667  123166.666667   
              Prichard                    NaN            NaN            NaN   
              Theodore           76900.000000   78300.000000   78066.666667   
              Daphne            123166.666667  120233.333333  122300.000000   
              Homewood          158333.333333  159666.666667  161100.000000   
...                                       ...            ...            ...   
Wisconsin     Holland           151033.333333  150500.000000  153233.333333   
West Virginia Parkersburg        54900.000000   55833.333333   56633.333333   
              Charleston         59533.333333   59366.666667   59366.666667   
              Elkview                     NaN            NaN            NaN   
              South Charleston            NaN            NaN            NaN   
              Vienna             72266.666667   74533.333333   75333.333333   
              Cross Lanes        78133.333333   78066.666667   79233.333333   
              Saint Albans       63833.333333   64700.000000   65200.000000   
              Dunbar                      NaN            NaN            NaN   
              Nitro                       NaN            NaN            NaN   
              Williamstown       82933.333333   85200.000000   86466.666667   
              Clendenin          39966.666667   38200.000000   40500.000000   
              Sissonville        57200.000000   58200.000000   61200.000000   
              Washington         83900.000000   86033.333333   88333.333333   
              Davisville                  NaN            NaN            NaN   
              Mineral Wells      75533.333333   77233.333333   81966.666667   
              Upper Falls        53033.333333   52333.333333   52100.000000   
              Chesapeake                  NaN            NaN            NaN   
          

In [16]:
def run_ttest():
    from scipy import stats
    x = get_list_of_university_towns()
    x['flg_ut'] = 'ut'

    y = convert_housing_data_to_quarters()
    y.reset_index(inplace = True)
    z = pd.merge(y, x, how='left', left_on=['State','RegionName'], right_on=['State','RegionName'])
    z['price_ratio'] = z['2008q2']/z['2009q2']

    smpl_1 = np.array(z.loc[z['flg_ut'] == 'ut']['price_ratio'].dropna())
    smpl_2 = np.array(z.loc[z['flg_ut'] != 'ut']['price_ratio'].dropna())
    t, p = ttest_ind(smpl_1, smpl_2)

    myf = lambda k : 'university town' if k < smpl_2.mean() else 'non-university town'
    return (p < 0.01, p, myf(smpl_1.mean()))
run_ttest()

(True, 0.0027134586648562414, 'university town')